In [ ]:
# Denis Shirvaey youtube channel b/w to color 

# Data Aug
# Medical Applications
# Semi -seupervised GANs

# GANS ?
A calss of ML techniques that consists of two networks playing an adversarial game against each other.
-Generator
-Detector
# Loss Function

In [9]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

In [10]:
class Discriminator(nn.Module):
    def __init__(self, img_dim):
        super().__init__()
        self.disc = nn.Sequential(
            nn.Linear(img_dim, 128),
            nn.LeakyReLU(0.1),
            nn.Linear(128, 1),
            nn.Sigmoid(),
        )
    def forward(self, x):
        return self.disc(x)

class Generator(nn.Module):
    def __init__(self, z_dim, img_dim):
        super().__init__()
        self.gen = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.LeakyReLU(0.1),
            nn.Linear(256, img_dim),
            nn.Tanh(),
        )
    def forward(self, x):
        return self.gen(x)


In [11]:
device = torch.device('cuda')

In [12]:
lr = 3e-4# best learning rate said by Andrej Karpathy for Adam
z_dim = 64# gans are sensitive to hyperparametrs in this case
img_dim = 28*28*1
batch_size =32
num_epochs = 50

In [13]:
disc = Discriminator(img_dim).to(device)
gen = Generator(z_dim, img_dim).to(device)
fixed_noise = torch.randn((batch_size, z_dim)).to(device)
transforms = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
)
dataset = datasets.MNIST(root = 'dataset/', transform=transforms, download = True)
loader = DataLoader(dataset, batch_size=batch_size, shuffle = True)
opt_disc = optim.Adam(disc.parameters(), lr = lr)
opt_gen = optim.Adam(gen.parameters(), lr = lr)
criterion = nn.BCELoss()
writer_fake = SummaryWriter(f"runs/GAN_MNIST/fake")
writer_real = SummaryWriter(f"runs/GAN_MNIST/real")
step = 0

In [27]:
!tensorboard --logdir='runs/GAN_MNIST/fake' --host localhost --port 8088

2020-12-30 14:56:16.484922: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
TensorBoard 2.4.0 at http://localhost:8088/ (Press CTRL+C to quit)
^C


In [22]:



!tensorboard --logdir logs/tensorboard

2020-12-30 14:51:13.164208: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.4.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C


In [17]:
!tensorboard --bind_all --port 8088 --logdir ./logs

2020-12-30 14:47:00.043193: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
TensorBoard 2.4.0 at http://b3b38870a7e2:8088/ (Press CTRL+C to quit)
^C


In [28]:
for epoch in range(num_epochs):
    for batch_idx, (real, _) in enumerate(loader):
        real = real.view(-1, 784).to(device)
        batch_size = real.shape[0]

        noise = torch.randn(batch_size, z_dim).to(device)
        fake = gen(noise)
        disc_real = disc(real).view(-1)
        lossD_real = criterion(disc_real, torch.ones_like(disc_real))
        disc_fake = disc(fake).view(-1) # fake.detach()
        lossD_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
        lossD = (lossD_real + lossD_fake)/2
        disc.zero_grad()
        lossD.backward(retain_graph =True)
        opt_disc.step()

        output = disc(fake).view(-1)
        lossG = criterion(output, torch.ones_like(output))
        gen.zero_grad()
        lossG.backward()
        opt_gen.step()
        if batch_idx == 0:
            print(
                f"Epoch [{epoch}/{num_epochs}] Batch {batch_idx}/{len(loader)} \
                      Loss D: {lossD:.4f}, loss G: {lossG:.4f}"
            )

            with torch.no_grad():
                fake = gen(fixed_noise).reshape(-1, 1, 28, 28)
                data = real.reshape(-1, 1, 28, 28)
                img_grid_fake = torchvision.utils.make_grid(fake, normalize=True)
                img_grid_real = torchvision.utils.make_grid(data, normalize=True)

                writer_fake.add_image(
                    "Mnist Fake Images", img_grid_fake, global_step=step
                )
                writer_real.add_image(
                    "Mnist Real Images", img_grid_real, global_step=step
                )
                step += 1

Epoch [0/50] Batch 0/1875                       Loss D: 0.0492, loss G: 4.0868
Epoch [1/50] Batch 0/1875                       Loss D: 0.0112, loss G: 5.2670
Epoch [2/50] Batch 0/1875                       Loss D: 0.0105, loss G: 4.7350
Epoch [3/50] Batch 0/1875                       Loss D: 0.0118, loss G: 5.0453
Epoch [4/50] Batch 0/1875                       Loss D: 0.0082, loss G: 4.9246
Epoch [5/50] Batch 0/1875                       Loss D: 0.0428, loss G: 4.2810
Epoch [6/50] Batch 0/1875                       Loss D: 0.0061, loss G: 6.4411
Epoch [7/50] Batch 0/1875                       Loss D: 0.0116, loss G: 5.6784
Epoch [8/50] Batch 0/1875                       Loss D: 0.0044, loss G: 6.4064
Epoch [9/50] Batch 0/1875                       Loss D: 0.0203, loss G: 5.6259
Epoch [10/50] Batch 0/1875                       Loss D: 0.0093, loss G: 5.8809
Epoch [11/50] Batch 0/1875                       Loss D: 0.0184, loss G: 6.7507
Epoch [12/50] Batch 0/1875                       L